In [1]:
# Import statements
import numpy as np
import random as rand
import os
import pandas
import time

In [2]:
# This cell contains all Connect4 related functions.

# This is a wrapper class that holds the logic engine and returns a decision.
class logic_engine_wrapper:

    # Defaults logic_engine to 0 so that it will just move over one space each turn.
    def __init__(self,logic_engine=0):
        self.logic_engine = logic_engine

    # Returns the logic_engine's move based on the provided board.
    def get_move(self,board):
        if type(self.logic_engine) is int:
            if self.logic_engine >= 6:
                self.logic_engine = 0
                return 6
            else:
                self.logic_engine += 1
                return self.logic_engine - 1
        else:
            output = self.logic_engine.get_output_vector(board.get_board_vector())
            greatest = 0
            for i in range(len(output)):
                if output[i] > output[greatest]:
                    greatest = i
            return greatest
                    
# This class represents a Connect4 board and contains all necessary utilities for it.
# On boards player1 = 1, player2 = -1, and blank = 0.
# The higher the first index the higher on the board i.e. index [5][6] is the top right corner of the board.
class game_board:

    # Creates a 7 x 6 game board.
    def __init__(self):
        self.board = [[0.0 for _ in range(7)] for _ in range(6)]

    # Prints the current game board.
    def print_board(self,player1_symbol='@',player2_symbol='#',clear=False):
        if clear:
            os.system('cls')
        for row in self.board[::-1]:
            print('|',end='')
            for num in row:
                symbol = player1_symbol if num == 1 else (player2_symbol if num == -1 else ' ')
                print(symbol,end='')
            print('|')
        print('---------')
        print(' 1234567 ')

    # Drops a piece into the indicated slot using a range of [0,6]
    def drop_piece(self,slot,player):
        if slot > 6 or slot < 0:
            return False
        else:
            for i in range(6):
                if self.board[i][slot] == 0:
                    self.board[i][slot] = player
                    return True
            return False

    # Returns the board squashed into a numpy array. (This is a utility function for net use)
    def get_board_vector(self):
        board_vector = []
        for row in self.board:
            board_vector += row
        return np.array(board_vector)

    # Returns a player if one of them as won otherwise returns 0.
    def get_winner(self):
        # Checks rows.
        for row in self.board:
            previous = None
            count = 0
            for num in row:
                if num == previous:
                    count += 1
                else:
                    previous = num
                    count = 1
                if count >= 4 and previous != 0:
                    return previous
        # Checks columns.
        for x in range(7):
            previous = None
            count = 0
            for y in range(6):
                num = self.board[y][x]
                if num == previous:
                    count += 1
                else:
                    previous = num
                    count = 1
                if count >= 4 and previous != 0:
                    return previous
        # Checks diagonals.
        for y in range(6):
            for x in range(7):
                num = self.board[y][x]
                if num != 0 and y < 3:
                    if x < 4:
                        if num == self.board[y+1][x+1] and num == self.board[y+2][x+2] and num == self.board[y+3][x+3]:
                            return num
                    elif x > 2:
                        if num == self.board[y+1][x-1] and num == self.board[y+2][x-2] and num == self.board[y+3][x-3]:
                            return num

        return 0


# This function enables the user to play against a logic_engine or lets two logic_engines play against each other.
def play_game(logic_engines=[logic_engine_wrapper()],verbose=True,ai_delay=0.2):
    board = game_board()
    current_player = 1 if rand.choice([True,False]) else -1
    max_num_of_turns = 6 * 7
    for turn in range(max_num_of_turns):
        if verbose:
            board.print_board(clear=True)
        if len(logic_engines) == 2:
            if verbose:
                print("Thinking...")
            time.sleep(ai_delay)
            slot = None
            if current_player == 1:
                proxy_board = game_board()
                for y in range(len(proxy_board.board)):
                    for x in range(len(proxy_board.board[y])):
                        proxy_board.board[y][x] = -1 * board.board[y][x]
                slot = logic_engines[0].get_move(proxy_board)
            else:
                slot = logic_engines[1].get_move(board)
            board.drop_piece(slot,current_player)
        else:
            if current_player == 1:
                slot = int(input('Input a slot: ')) - 1
                board.drop_piece(slot,current_player)
            else:
                if verbose:
                    print("Thinking...")
                time.sleep(ai_delay)
                slot = logic_engines[0].get_move(board)
                board.drop_piece(slot,current_player)
        winner = board.get_winner()
        if winner != 0:
            if verbose:
                board.print_board(clear=True)
            return winner
        else:
            current_player = 1 if current_player == -1 else -1
        if turn == (6 * 7) - 1 and verbose:
            board.print_board(clear=True)
    return 0

In [3]:
#Non-linear functions are declared in this cell.

#Sigmoid function.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
# Info String Seperators:
net_section_seperator = '|'
net_sub_section_seperator = ':' # i.e. each hidden layer and each bias layer
net_matrix_row_seperator = '>'
net_vector_index_seperator = ','

In [5]:
# Feedforward network class that's used.
class net:
    
    # wiki_net class constructor.
    def __init__(self,input_size,hidden_layers,non_linears=sigmoid,use_biases=True,name="N/A"):
        self.hidden_layers = []
        self.non_linears = []
        self.biases = []
        self.name = name
        if type(non_linears) != list:
            non_linears = [non_linears] * len(hidden_layers) 
            previous_size = input_size
        elif len(non_linears) == 1:
            non_linears *= len(hidden_layers)
        for i in range(len(hidden_layers)):
            self.hidden_layers.append(2 * np.random.random((previous_size,hidden_layers[i])) - 1)
            self.non_linears.append(non_linears[i])
            self.biases.append(2 * np.random.random(hidden_layers[i]) - 1 if use_biases else np.zeros(hidden_layers[i]))
            previous_size = hidden_layers[i]
    
    # Gets the output vector for a given input vector.
    def get_output_vector(self,input_vector):
        output_vector = input_vector
        for i in range(len(self.hidden_layers)):
            output_vector = self.non_linears[i](np.dot(output_vector,self.hidden_layers[i] + self.biases[i]))
        return output_vector
    
    # Returns a string that contains all of the network's features in the following order, [Name, Score, Hidden_Layers, Biases]
    def generate_info_string(self,score=0):
        info_string = f'{self.name}{net_section_seperator}{score}{net_section_seperator}'
        count_1 = 0
        count_2 = 0
        count_3 = 0
        for layer in self.hidden_layers:
            for vector in layer:
                for index in vector:
                    info_string += f'{index}'
                    info_string += net_vector_index_seperator if count_1 != len(vector) - 1 else ''
                    count_1 += 1
                count_1 = 0
                info_string += net_matrix_row_seperator if count_2 != len(layer) - 1 else ''
                count_2 += 1
            count_2 = 0
            info_string += net_sub_section_seperator if count_3 != len(self.hidden_layers) - 1 else ''
            count_3 += 1
        info_string += net_section_seperator
        count_1 = 0
        count_2 = 0
        for bias in self.biases:
            for index in bias:
                info_string += f'{index}'
                info_string += net_vector_index_seperator if count_1 != len(bias) - 1 else ''
                count_1 += 1
            count_1 = 0
            info_string += net_sub_section_seperator if count_2 != len(self.biases) - 1 else ''
            count_2 += 1
        return info_string
    
    # Loads the net's attribute's from a given string and returns it's score.
    # String Format is listed above the generate_info_string method
    def load_info_from_string(self,info_string,non_linear=sigmoid):
        sections = info_string.split(net_section_seperator)
        self.name = sections[0]
        score = float(sections[1])
        self.hidden_layers = []
        self.biases = []
        self.non_linears = [non_linear]
        hidden_layer_sections = sections[2].split(net_sub_section_seperator)
        for layer in hidden_layer_sections:
            vectors = []
            for text_vector in layer.split(net_matrix_row_seperator):
                vector = []
                for index in text_vector.split(net_vector_index_seperator):
                    vector.append(float(index))
                vectors.append(vector)
            self.hidden_layers.append(np.array(vectors))
        self.non_linears *= len(self.hidden_layers)
        text_biases = sections[3].split(net_sub_section_seperator)
        for text_bias in text_biases:
            bias = []
            for index in text_bias.split(net_vector_index_seperator):
                bias.append(float(index))
            self.biases.append(np.array(bias))
        return score

In [6]:
# All functions related to saving and loading data are in this cell.

# Saves the given wrapped net population.
def save_wrapped_net_population(population,name,generation):
    try:
        file = open(f'Populations/{name}.snp','w')
        file.write(f'Generation {generation}')
        for wrapped_net in population:
            file.write(f'\n{wrapped_net[0].generate_info_string(wrapped_net[1])}')
        file.close()
    except:
        print(f'Failed to save population : {name}')
        
# Loads a net population from a given file in format (population,generation)
def load_wrapped_net_population(name):
    file = None
    try:
        file = open(f'Populations/{name}.snp','r')
    except:
        print(f'Failed to find file for population : {name}')
        return None
    if file != None:
        contents = file.read().split('\n')
        generation = int(contents[0].split(' ')[1])
        population = []
        for val in contents[1:]:
            net = generate_standard_net()
            score = net.load_info_from_string(val)
            population.append((net,score))
        return (population,generation)

In [7]:
# Generates a net_net with preset values (this is basically a way to make net generation easier).
def generate_standard_net(name="N/A"): 
    return net(input_size=42,hidden_layers=[7],name=name)
    #return net(input_size=42,hidden_layers=[20,10,7],name=name)
    #return net_net(input_size=5,hidden_layers=[3,2,1],name=name)

In [8]:
# Genetic evolution related functions are declared in this cell.

# Breeds two nets using indexed addition and the specified weight, mates child with a random net if mutating.
def breed_nets(net_1,net_2,mutation=False,non_linear=sigmoid,name="N/A"):
    hidden_layers = []
    biases = []
    non_linears = []
    for i in range(len(net_1.hidden_layers)):
        #hidden_layers.append((net_1.hidden_layers[i] * net_1_weight) + (net_2.hidden_layers[i] * (1 - net_1_weight)))
        #biases.append((net_1.biases[i] * net_1_weight) + (net_2.biases[i] * (1 - net_1_weight)))
        non_linears.append(non_linear)
        hidden_layers.append([])
        biases.append([])
        for j in range(len(net_1.hidden_layers[i])):
            hidden_layers[i].append((rand.choice([net_1,net_2])).hidden_layers[i][j])
        hidden_layers[i] = np.array(hidden_layers[i])
        for j in range(len(net_1.biases[i])):
            biases[i].append((rand.choice([net_1,net_2])).biases[i][j])
        biases[i] = np.array(biases[i])
    child = generate_standard_net(name=name)
    child.hidden_layers = hidden_layers
    child.biases = biases
    child.non_linears = non_linears
    if mutation:
        return breed_nets(child,generate_standard_net(),name=name)
    else:
        return child

# Breeds a population of ordered wrapped nets.
def breed_wrapped_net_popultation(old_population,generation):
    avg_score = 0
    old_size = len(old_population)
    for wrapped_net in old_population:
        avg_score += wrapped_net[1]
    avg_score /= len(old_population)
    new_population = []
    rand.shuffle(old_population)
    for wrapped_net in old_population:
        if wrapped_net[1] >= avg_score and len(new_population) < len(old_population) / 2:
            new_population.append(wrapped_net)
    old_population = new_population
    new_population = []
    child_num = 0
    rand.shuffle(old_population)
    while len(old_population) > 1:
        wrapped_nets = [old_population.pop(),old_population.pop()]
        score_sum = wrapped_nets[0][1] + wrapped_nets[1][1]
        child_num += 1
        child_1 = breed_nets(wrapped_nets[0][0],wrapped_nets[1][0],mutation=np.random.random(1)[0] > 0.95,name=f'Net : {child_num} (Generation : {generation})')
        child_num += 1
        child_2 = breed_nets(wrapped_nets[0][0],wrapped_nets[1][0],mutation=np.random.random(1)[0] > 0.95,name=f'Net : {child_num} (Generation : {generation})')
        new_population.append((child_1,0))
        new_population.append((child_2,0))
        if avg_score <= 0:
            if np.random.random(1)[0] > 0.5:
                new_population = add_wrapped_net(new_population,wrapped_nets[0])
            else:
                new_population = add_wrapped_net(new_population,wrapped_nets[1])
        else:
            for wrapped_net in wrapped_nets:
                if wrapped_net[1] >= avg_score:
                        new_population = add_wrapped_net(new_population,wrapped_net)
    if len(old_population) <= 3:
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    while len(old_population) > 0:
        new_population = add_wrapped_net(new_population,old_population.pop())
    if len(new_population) <= 0:
        for _ in range(8):
            child_num += 1
            new_population.append((generate_standard_net_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    while len(new_population) < old_size:
        child_num += 1
        new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
   # if generation % 4:
           # child_num += 1
           # new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    if generation % 200 == 0:
        new_population = new_population[:20]
        for _ in range(25):
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    return new_population
    
# Subjects a population to natural selection for the given number of generations.
def subject_popultaion_to_natural_selection(population,starting_generation,name,num_of_generations=100,verbose_crawl=False):
    for i in range(num_of_generations):
        current_generation = starting_generation + i
        print(f'Beginning Natural Selection For Generation : {current_generation} | Population Size : {len(population)}')
        new_population = []
        high_score = 0
        count = 0
        start = time.time()
        for wrapped_net in population:
            new_population.append((wrapped_net[0],0))
        for i in range(len(new_population) - 1):
            for j in range(len(new_population) - i):
                if j != 0:
                    logic_engine_1 = logic_engine_wrapper(logic_engine=population[i][0])
                    logic_engine_2 = logic_engine_wrapper(logic_engine=population[i+j][0])
                    result = play_game(logic_engines=[logic_engine_1,logic_engine_2],verbose=False,ai_delay=0)
                    if result == 1:
                        new_population[i] = (new_population[i][0],new_population[i][1]+1)
                    elif result == -1:
                        new_population[i] = (new_population[i+j][0],new_population[i+j][1]+1)
        proxy_pop = []
        for wrapped_net in new_population:
            add_wrapped_net(proxy_pop,wrapped_net)
            if wrapped_net[1] > high_score:
                high_score = wrapped_net[1]
        new_population = proxy_pop
        end = time.time()
        print(f'Generation {current_generation} High Score : {high_score} | Duration : {end - start} Secs')
        population = breed_wrapped_net_popultation(new_population,current_generation + 1)
        save_wrapped_net_population(population,name,current_generation + 1)
        print(f'Generation {current_generation + 1} Bred')

# Generates a starting population of nets. The nets are in a tuple wrapper of format (net,score)
def generate_base_wrapped_net_population(size=100):
    return [(generate_standard_net(f'Net : {i + 1} (Generation : 0)'),0) for i in range(size)]

# Adds a net wrapper to the given array in order of greatest score.
def add_wrapped_net(array,wrapped_net):
    if len(array) == 0:
        array.append(wrapped_net)
    else:
        placed = False
        for i in range(len(array)):
            if array[i][1] < wrapped_net[1] and not placed:
                array.insert(i,wrapped_net)
                placed = True
        if not placed:
            array.append(wrapped_net)
    return array

In [9]:
# Creates a new population.
population = generate_base_wrapped_net_population(10)
generation = 0

In [10]:
# Loads a population from the disk.
hold = load_wrapped_net_population('eigthTrial')
population = hold[0]
generation = hold[1]

In [11]:
subject_popultaion_to_natural_selection(population,generation,name='eigthTrial',num_of_generations=50000,verbose_crawl=False)

Beginning Natural Selection For Generation : 40128 | Population Size : 119
Generation 40128 High Score : 11 | Duration : 3.7552502155303955 Secs
Generation 40129 Bred
Beginning Natural Selection For Generation : 40129 | Population Size : 123
Generation 40129 High Score : 6 | Duration : 3.797333240509033 Secs
Generation 40130 Bred
Beginning Natural Selection For Generation : 40130 | Population Size : 123
Generation 40130 High Score : 10 | Duration : 3.6103763580322266 Secs
Generation 40131 Bred
Beginning Natural Selection For Generation : 40131 | Population Size : 123
Generation 40131 High Score : 9 | Duration : 4.218008995056152 Secs
Generation 40132 Bred
Beginning Natural Selection For Generation : 40132 | Population Size : 123
Generation 40132 High Score : 11 | Duration : 5.165334463119507 Secs
Generation 40133 Bred
Beginning Natural Selection For Generation : 40133 | Population Size : 123
Generation 40133 High Score : 7 | Duration : 4.092949151992798 Secs
Generation 40134 Bred
Begin

Generation 40178 Bred
Beginning Natural Selection For Generation : 40178 | Population Size : 163
Generation 40178 High Score : 23 | Duration : 6.979217529296875 Secs
Generation 40179 Bred
Beginning Natural Selection For Generation : 40179 | Population Size : 163
Generation 40179 High Score : 7 | Duration : 8.028745412826538 Secs
Generation 40180 Bred
Beginning Natural Selection For Generation : 40180 | Population Size : 163
Generation 40180 High Score : 9 | Duration : 9.371948480606079 Secs
Generation 40181 Bred
Beginning Natural Selection For Generation : 40181 | Population Size : 163
Generation 40181 High Score : 10 | Duration : 7.631445407867432 Secs
Generation 40182 Bred
Beginning Natural Selection For Generation : 40182 | Population Size : 163
Generation 40182 High Score : 13 | Duration : 7.710837364196777 Secs
Generation 40183 Bred
Beginning Natural Selection For Generation : 40183 | Population Size : 163
Generation 40183 High Score : 12 | Duration : 9.931437015533447 Secs
Genera

Generation 40228 High Score : 6 | Duration : 1.851060152053833 Secs
Generation 40229 Bred
Beginning Natural Selection For Generation : 40229 | Population Size : 75
Generation 40229 High Score : 6 | Duration : 2.0330607891082764 Secs
Generation 40230 Bred
Beginning Natural Selection For Generation : 40230 | Population Size : 75
Generation 40230 High Score : 7 | Duration : 1.2999708652496338 Secs
Generation 40231 Bred
Beginning Natural Selection For Generation : 40231 | Population Size : 75
Generation 40231 High Score : 10 | Duration : 2.059844493865967 Secs
Generation 40232 Bred
Beginning Natural Selection For Generation : 40232 | Population Size : 75
Generation 40232 High Score : 5 | Duration : 1.7633247375488281 Secs
Generation 40233 Bred
Beginning Natural Selection For Generation : 40233 | Population Size : 79
Generation 40233 High Score : 5 | Duration : 1.401700496673584 Secs
Generation 40234 Bred
Beginning Natural Selection For Generation : 40234 | Population Size : 79
Generation 4

Generation 40278 High Score : 7 | Duration : 2.2418785095214844 Secs
Generation 40279 Bred
Beginning Natural Selection For Generation : 40279 | Population Size : 88
Generation 40279 High Score : 6 | Duration : 2.241421937942505 Secs
Generation 40280 Bred
Beginning Natural Selection For Generation : 40280 | Population Size : 88
Generation 40280 High Score : 14 | Duration : 2.220095157623291 Secs
Generation 40281 Bred
Beginning Natural Selection For Generation : 40281 | Population Size : 88
Generation 40281 High Score : 10 | Duration : 2.5097997188568115 Secs
Generation 40282 Bred
Beginning Natural Selection For Generation : 40282 | Population Size : 88
Generation 40282 High Score : 10 | Duration : 2.280604600906372 Secs
Generation 40283 Bred
Beginning Natural Selection For Generation : 40283 | Population Size : 88
Generation 40283 High Score : 8 | Duration : 2.6560747623443604 Secs
Generation 40284 Bred
Beginning Natural Selection For Generation : 40284 | Population Size : 88
Generation

Generation 40328 High Score : 8 | Duration : 3.207094430923462 Secs
Generation 40329 Bred
Beginning Natural Selection For Generation : 40329 | Population Size : 107
Generation 40329 High Score : 6 | Duration : 3.636129140853882 Secs
Generation 40330 Bred
Beginning Natural Selection For Generation : 40330 | Population Size : 107
Generation 40330 High Score : 12 | Duration : 2.8842341899871826 Secs
Generation 40331 Bred
Beginning Natural Selection For Generation : 40331 | Population Size : 111
Generation 40331 High Score : 4 | Duration : 2.647148847579956 Secs
Generation 40332 Bred
Beginning Natural Selection For Generation : 40332 | Population Size : 111
Generation 40332 High Score : 11 | Duration : 3.833623170852661 Secs
Generation 40333 Bred
Beginning Natural Selection For Generation : 40333 | Population Size : 111
Generation 40333 High Score : 9 | Duration : 3.8286211490631104 Secs
Generation 40334 Bred
Beginning Natural Selection For Generation : 40334 | Population Size : 111
Genera

...

Generation 43077 High Score : 8 | Duration : 3.8246912956237793 Secs
Generation 43078 Bred
Beginning Natural Selection For Generation : 43078 | Population Size : 107
Generation 43078 High Score : 9 | Duration : 3.8296120166778564 Secs
Generation 43079 Bred
Beginning Natural Selection For Generation : 43079 | Population Size : 107
Generation 43079 High Score : 18 | Duration : 3.6927554607391357 Secs
Generation 43080 Bred
Beginning Natural Selection For Generation : 43080 | Population Size : 107
Generation 43080 High Score : 11 | Duration : 3.432851791381836 Secs
Generation 43081 Bred
Beginning Natural Selection For Generation : 43081 | Population Size : 111
Generation 43081 High Score : 8 | Duration : 3.802295684814453 Secs
Generation 43082 Bred
Beginning Natural Selection For Generation : 43082 | Population Size : 115
Generation 43082 High Score : 8 | Duration : 3.2408604621887207 Secs
Generation 43083 Bred
Beginning Natural Selection For Generation : 43083 | Population Size : 115
Gene

Generation 43127 High Score : 6 | Duration : 5.471860885620117 Secs
Generation 43128 Bred
Beginning Natural Selection For Generation : 43128 | Population Size : 135
Generation 43128 High Score : 9 | Duration : 4.9679296016693115 Secs
Generation 43129 Bred
Beginning Natural Selection For Generation : 43129 | Population Size : 135
Generation 43129 High Score : 17 | Duration : 5.57201623916626 Secs
Generation 43130 Bred
Beginning Natural Selection For Generation : 43130 | Population Size : 135
Generation 43130 High Score : 8 | Duration : 6.474775075912476 Secs
Generation 43131 Bred
Beginning Natural Selection For Generation : 43131 | Population Size : 135
Generation 43131 High Score : 9 | Duration : 5.115737199783325 Secs
Generation 43132 Bred
Beginning Natural Selection For Generation : 43132 | Population Size : 135
Generation 43132 High Score : 6 | Duration : 6.719831943511963 Secs
Generation 43133 Bred
Beginning Natural Selection For Generation : 43133 | Population Size : 135
Generatio

Generation 43177 High Score : 10 | Duration : 7.480166435241699 Secs
Generation 43178 Bred
Beginning Natural Selection For Generation : 43178 | Population Size : 159
Generation 43178 High Score : 9 | Duration : 9.746390342712402 Secs
Generation 43179 Bred
Beginning Natural Selection For Generation : 43179 | Population Size : 159
Generation 43179 High Score : 10 | Duration : 10.350017786026001 Secs
Generation 43180 Bred
Beginning Natural Selection For Generation : 43180 | Population Size : 163
Generation 43180 High Score : 7 | Duration : 7.933473587036133 Secs
Generation 43181 Bred
Beginning Natural Selection For Generation : 43181 | Population Size : 163
Generation 43181 High Score : 14 | Duration : 8.808972120285034 Secs
Generation 43182 Bred
Beginning Natural Selection For Generation : 43182 | Population Size : 163
Generation 43182 High Score : 12 | Duration : 7.378990650177002 Secs
Generation 43183 Bred
Beginning Natural Selection For Generation : 43183 | Population Size : 163
Gener

Generation 43227 High Score : 10 | Duration : 1.0430908203125 Secs
Generation 43228 Bred
Beginning Natural Selection For Generation : 43228 | Population Size : 59
Generation 43228 High Score : 5 | Duration : 1.1824557781219482 Secs
Generation 43229 Bred
Beginning Natural Selection For Generation : 43229 | Population Size : 59
Generation 43229 High Score : 8 | Duration : 1.156670093536377 Secs
Generation 43230 Bred
Beginning Natural Selection For Generation : 43230 | Population Size : 59
Generation 43230 High Score : 13 | Duration : 1.1581599712371826 Secs
Generation 43231 Bred
Beginning Natural Selection For Generation : 43231 | Population Size : 59
Generation 43231 High Score : 6 | Duration : 1.2047829627990723 Secs
Generation 43232 Bred
Beginning Natural Selection For Generation : 43232 | Population Size : 59
Generation 43232 High Score : 7 | Duration : 1.0629394054412842 Secs
Generation 43233 Bred
Beginning Natural Selection For Generation : 43233 | Population Size : 60
Generation 4

KeyboardInterrupt: 

In [73]:
population = load_wrapped_net_population('eigthTrial')[0]

In [18]:
population  = population[:30]
len(population)

30

In [32]:
logic_engine_1 = logic_engine_wrapper(logic_engine=population[3][0])
logic_engine_2 = logic_engine_wrapper(logic_engine=population[5][0])
result = play_game(logic_engines=[logic_engine_1,logic_engine_2],verbose=True,ai_delay=0)

|       |
|       |
|       |
|       |
|       |
|       |
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|       |
|#      |
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|       |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|#      |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#    #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#@   #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|##     |
|#@   #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|##     |
|#@@  #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@#     |
|##     |
|#@@  #@|
---------
 1234567 
Thinking...
|       |
|       |
|@      |
|@#     |
|##     |
|#@@  #@|
---------
 1234567 


In [14]:
len(generate_standard_net().hidden_layers[0])

42